In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [8]:
# 1. Load Data
ptb_normal = pd.read_csv('D:\\mlmed2026\\dataset\\ptbdb_normal.csv', header=None)
ptb_abnormal = pd.read_csv('D:\\mlmed2026\\dataset\\ptbdb_abnormal.csv', header=None)

In [9]:
#2. Preprocessing
# Combine and Shuffle
ptb_full = pd.concat([ptb_normal, ptb_abnormal], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

# Split features (0-186) and label (187)
X = ptb_full.iloc[:, :-1].values
y = ptb_full.iloc[:, -1].values

# Reshape for 1D CNN: [samples, time_steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [10]:
# 3. Build 1D CNN Model
model = models.Sequential([
    layers.Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(187, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\Minh Duc\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# 4. Train
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8297 - loss: 0.3788 - val_accuracy: 0.7313 - val_loss: 0.7920
Epoch 2/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9173 - loss: 0.2042 - val_accuracy: 0.8275 - val_loss: 0.3843
Epoch 3/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9427 - loss: 0.1462 - val_accuracy: 0.9511 - val_loss: 0.1224
Epoch 4/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9575 - loss: 0.1083 - val_accuracy: 0.8129 - val_loss: 0.8315
Epoch 5/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9658 - loss: 0.0922 - val_accuracy: 0.7785 - val_loss: 1.0555
Epoch 6/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9764 - loss: 0.0618 - val_accuracy: 0.7588 - val_loss: 1.6179
Epoch 7/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9802 - loss: 0.0540 - val_accuracy: 0.7880 - val_loss: 1.0317
Epoch 8/10
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9838 - loss: 0.0432 - val_accuracy: 0.

In [12]:
# 5. Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9787 - loss: 0.0817
Test Accuracy: 0.9787
